In [0]:
from pyspark.sql.functions import  *
from pyspark.sql.types import *

## Parameters

In [0]:
# # key columns
# dbutils.widgets.text("keycols","['flight_id']")

# # CDC Column
# dbutils.widgets.text("cdcol","")

# #back-date refresh
# dbutils.widgets.text("backdated_refresh","")

# #Source Object
# dbutils.widgets.text("source_object","")

# # Source Schema
# dbutils.widgets.text("source_schema","")


## Fetching Parameters & creating variables

In [0]:
# # Key Cols List
# key_cols_list = eval( dbutils.widgets.get("keycols"))

# #CDC Column
# cdc_col = dbutils.widgets.get("cdcol")

# # Backdated refresh
# backdated_refresh = dbutils.widgets.get("backdated_refresh")


# # Source Object
# source_object = dbutils.widgets.get("source_object")

# # Source Schema
# source_schema = dbutils.widgets.get("source_schema")

In [0]:
# dbutils.widgets.remove("backdated refresh")
# dbutils.widgets.remove("backdate refresh")

## Parameters

In [0]:
#Flights

# #Catalog Name
# catalog = "workspace"

# # Key Cols List
# key_cols = "['flight_id']"
# key_cols_list = eval(key_cols)

# #CDC Column
# cdc_col = "modifiedDate"

# # Backdated refresh
# backdated_refresh = ""

# # Source Object
# source_object = "silver_flights"

# # Source Schema
# source_schema = "silver"

# #Tareget Schema
# target_schema = "gold"

# #Target Object
# target_object = "DimFlights"

# #Surrogate key
# surrogate_key = "DimFlightsKey"

In [0]:
# # Airports

# # Catalog Name
# catalog = "workspace"

# # Key Cols List
# key_cols = "['airport_id']"
# key_cols_list = eval(key_cols)

# # CDC Column
# cdc_col = "modifiedDate"

# # Backdated Refresh
# backdated_refresh = ""

# # Source Object
# source_object = "silver_airports"

# # Source Schema
# source_schema = "silver"

# # Target Schema 
# target_schema = "gold"

# # Target Object 
# target_object = "DimAirports"

# # Surrogate Key
# surrogate_key = "DimAirportsKey"


In [0]:
# Catalog Name
catalog = "workspace"

# Key Cols List
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)

# CDC Column
cdc_col = "modifiedDate"

# Backdated Refresh
backdated_refresh = ""

# Source Object
source_object = "silver_passengers"

# Source Schema
source_schema = "silver"

# Target Schema 
target_schema = "gold"

# Target Object 
target_object = "DimPassengers"

# Surrogate Key
surrogate_key = "DimPassengersKey"


In [0]:
key_cols_list

# **Increamental Data Ingestion**

### Last Load Date

In [0]:
#No Back Dated Refresh
if len(backdated_refresh) == 0:

    # If Table Exists In The Destination
    if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

        last_load = spark.sql(f"select max({cdc_col}) from {catalog}.{target_schema}.{target_object}").collect()[0][0]

    else:
        last_load = "1900-01-01 00:00:00"

# Yes Back_Dated Refresh
else:
    last_load = backdated_refresh

#Test The Last Load
last_load

In [0]:
df_src = spark.sql(f"select * from {source_schema}.{source_object} where {cdc_col} > '{last_load}' ")

In [0]:
df_src.display()

#### OLD vs NEW Records

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    # Key Column String for increamental
    key_cols_string_incremental = ", ".join(key_cols_list)

    df_trg = spark.sql(f"""
                       select {key_cols_string_incremental}, {surrogate_key}, create_date , update_date 
                       from {catalog}.{target_schema}.{target_object}
                       """)

else:
    # key columns string for initial
    key_cols_string_init = [f" '' as {i} " for i in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)
    
    df_trg = spark.sql(f"""
                       select {key_cols_string_init},CAST('0' AS INT) as {surrogate_key}, cast('1900-01-01 00:00:00' as timestamp) as create_date ,cast('1900-01-01 00:00:00' as timestamp) as update_date where 1=0
                       """)

In [0]:
df_trg.display()

## join condition

In [0]:
' AND '.join([f"src.{i} = trg.{i}" for i in key_cols_list])

In [0]:
join_condition = ' AND '.join([f"src.{i} = trg.{i}" for i in key_cols_list])

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

In [0]:
spark.sql("select * from src where 1=0").show()

In [0]:
spark.sql("select * from trg where 1=0").show()

In [0]:
df_join = spark.sql(
    f"""
    select 
        src.*,
        trg.{surrogate_key},
        trg.create_date,
        trg.update_date
    from src
    left join trg
    on {join_condition}
    """
)
display(df_join)

In [0]:
# OLD Records
df_old = df_join.filter(col(f"trg.{surrogate_key}").isNotNull())
# New Records
df_new = df_join.filter(col(f"trg.{surrogate_key}").isNull())

In [0]:
display(df_old)

In [0]:
display(df_new)

## **ENRICHING DFS**

#### **Preparing DF_OLD**

In [0]:
df_old_enr = df_old = df_old.withColumn("update_date", current_timestamp())

In [0]:
df_old_enr.display()

#### **Preparing DF_NEW**

In [0]:
df_new.display()

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(f"""
                                    select max({surrogate_key}) from {catalog}.{target_schema}.{target_object}
                                    """).collect()[0][0]
    
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                .withColumn("create_date", current_timestamp())\
                .withColumn("update_date", current_timestamp())

else:
    max_surrogate_key = 1
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                .withColumn("create_date", current_timestamp())\
                .withColumn("update_date", current_timestamp())


In [0]:
df_new_enr.display()

## **Unioning OLD AND NEW RECORDS**

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)

In [0]:
df_union.display()

## **UPSERT**

In [0]:
from delta.tables import DeltaTable 

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    dlt_obj = DeltaTable.forName(spark, f'{catalog}.{target_schema}.{target_object}')
    dlt_obj.alias("trg").merge(
        df_union.alias("src"), f"trg.{surrogate_key} = src.{surrogate_key}")\
        .whenMatchedUpdateAll(condition = f"src.{cdc_col} >= trg.{cdc_col}")\
        .whenNotMatchedInsertAll()\
        .execute()

else: 

    df_union.write.format("delta")\
            .mode("append")\
            .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
show tables in gold;

In [0]:
%sql
select * from workspace.gold.dimpassengers where passenger_id = 'P0049'